### Grading
The final score that you will receive for your programming assignment is generated in relation to the total points set in your programming assignment item—not the total point value in the nbgrader notebook.<br>
When calculating the final score shown to learners, the programming assignment takes the percentage of earned points vs. the total points provided by nbgrader and returns a score matching the equivalent percentage of the point value for the programming assignment. <br>
**DO NOT CHANGE VARIABLE OR METHOD SIGNATURES** The autograder will not work properly if your change the variable or method signatures. 

### Validate Button
Please note that this assignment uses nbgrader to facilitate grading. You will see a **validate button** at the top of your Jupyter notebook. If you hit this button, it will run tests cases for the lab that aren't hidden. It is good to use the validate button before submitting the lab. Do know that the labs in the course contain hidden test cases. The validate button will not let you know whether these test cases pass. After submitting your lab, you can see more information about these hidden test cases in the Grader Output. <br>
***Cells with longer execution times will cause the validate button to time out and freeze. Please know that if you run into Validate time-outs, it will not affect the final submission grading.*** <br>

# Building Recommender Systems for Movie Rating Prediction

In this assignment, we will build a recommender systems that predict movie ratings. [MovieLense](https://grouplens.org/datasets/movielens/) has currently 25 million user-movie ratings.  Since the entire data is too big, we use  a 1 million ratings subset [MovieLens 1M](https://www.kaggle.com/odedgolden/movielens-1m-dataset), and we reformatted the data to make it more convenient to use.

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix, csr_matrix
from scipy.spatial.distance import jaccard, cosine 
from pytest import approx

In [23]:
MV_users = pd.read_csv('data/users.csv')
MV_movies = pd.read_csv('data/movies.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [24]:
from collections import namedtuple
Data = namedtuple('Data', ['users','movies','train','test'])
data = Data(MV_users, MV_movies, train, test)

In [25]:
print(data)

Data(users=       uID gender  age  accupation    zip
0        1      F    1          10  48067
1        2      M   56          16  70072
2        3      M   25          15  55117
3        4      M   45           7  02460
4        5      M   25          20  55455
...    ...    ...  ...         ...    ...
6035  6036      F   25          15  32603
6036  6037      F   45           1  76006
6037  6038      F   56           1  14706
6038  6039      F   45           0  01060
6039  6040      M   25           6  11106

[6040 rows x 5 columns], movies=       mID                        title  year  Doc  Com  Hor  Adv  Wes  Dra  \
0        1                    Toy Story  1995    0    1    0    0    0    0   
1        2                      Jumanji  1995    0    0    0    1    0    0   
2        3             Grumpier Old Men  1995    0    1    0    0    0    0   
3        4            Waiting to Exhale  1995    0    1    0    0    0    1   
4        5  Father of the Bride Part II  1995    0    1  

### Starter codes
Now, we will be building a recommender system which has various techniques to predict ratings. 
The `class RecSys` has baseline prediction methods (such as predicting everything to 3 or to average rating of each user) and other utility functions. `class ContentBased` and `class Collaborative` inherit `class RecSys` and further add methods calculating item-item similarity matrix. You will be completing those functions using what we learned about content-based filtering and collaborative filtering.

`RecSys`'s `rating_matrix` method converts the (user id, movie id, rating) triplet from the train data (train data's ratings are known) into a utility matrix for 6040 users and 3883 movies.    
Here, we create the utility matrix as a dense matrix (numpy.array) format for convenience. But in a real world data where hundreds of millions of users and items may exist, we won't be able to create the utility matrix in a dense matrix format (For those who are curious why, try measuring the dense matrix self.Mr using .nbytes()). In that case, we may use sparse matrix operations as much as possible and distributed file systems and distributed computing will be needed. Fortunately, our data is small enough to fit in a laptop/pc memory. Also, we will use numpy and scipy.sparse, which allow significantly faster calculations than calculating on pandas.DataFrame object.    
In the `rating_matrix` method, pay attention to the index mapping as user IDs and movie IDs are not the same as array index.

In [42]:
from sklearn.metrics import jaccard_score
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
class RecSys():
    def __init__(self,data):
        self.data=data
        self.allusers = list(self.data.users['uID'])
        self.allmovies = list(self.data.movies['mID'])
        self.genres = list(self.data.movies.columns.drop(['mID', 'title', 'year']))
        self.mid2idx = dict(zip(self.data.movies.mID,list(range(len(self.data.movies)))))
        self.uid2idx = dict(zip(self.data.users.uID,list(range(len(self.data.users)))))
        self.Mr=self.rating_matrix()
        self.Mm=None 
        self.sim=np.zeros((len(self.allmovies),len(self.allmovies)))
        
    def rating_matrix(self):
        """
        Convert the rating matrix to numpy array of shape (#allusers,#allmovies)
        """
        ind_movie = [self.mid2idx[x] for x in self.data.train.mID] 
        ind_user = [self.uid2idx[x] for x in self.data.train.uID]
        rating_train = list(self.data.train.rating)
        
        return np.array(coo_matrix((rating_train, (ind_user, ind_movie)), shape=(len(self.allusers), len(self.allmovies))).toarray())


    def predict_everything_to_3(self):
        """
        Predict everything to 3 for the test data
        """
        # Generate an array with 3s against all entries in test dataset
        # your code here
        a = np.empty(1)
        a.fill(3)
        return a
    
    def predict_to_user_average(self):
        """
        Predict to average rating for the user.
        Returns numpy array of shape (#users,)
        """
        # Generate an array as follows:
        # 1. Calculate all avg user rating as sum of ratings of user across all movies/number of movies whose rating > 0
        # 2. Return the average rating of users in test data
        # your code here
        test_users = [self.uid2idx[x] for x in self.data.test.uID]
        usr_avg = []
        for uid in test_users:
            col = self.Mr[uid]
            col = col[col > 0]
            usr_avg.append(np.mean(col))
        return np.array(usr_avg)
    
    def predict_from_sim(self,uid,mid):
        """
        Predict a user rating on a movie given userID and movieID
        """
        # Predict user rating as follows:
        # 1. Get entry of user id in rating matrix
        # 2. Get entry of movie id in sim matrix
        # 3. Employ 1 and 2 to predict user rating of the movie
        # your code here
        
        user_idx = self.uid2idx[uid]
        user_ratings = self.Mr[user_idx]
        movie_idx = self.mid2idx[mid]
        similarity_row = self.Mr[movie_idx]
        #print(user_ratings)
        #print(similarity_row)
        weighted_ratings = user_ratings * similarity_row
        total_weighted_ratings = np.sum(weighted_ratings)
        rated_movies = user_ratings > 0
        sum_of_similarities = np.sum(similarity_row[rated_movies])
        if sum_of_similarities != 0:
            predicted_rating = total_weighted_ratings / sum_of_similarities
        else:
            predicted_rating = np.mean(user_ratings[user_ratings > 0])
        return predicted_rating
    
    def predict(self):
        """
        Predict ratings in the test data. Returns predicted rating in a numpy array of size (# of rows in testdata,)
        """
        # your code here
        predicted_ratings = []
        for _, row in self.data.test.iterrows():
            uid = row['uID']
            mid = row['mID']
            predicted_rating = self.predict_from_sim(uid, mid)
            predicted_ratings.append(predicted_rating)
        return np.array(predicted_ratings)
        
    
    def rmse(self,yp):
        yp[np.isnan(yp)]=3 #In case there is nan values in prediction, it will impute to 3.
        yt=np.array(self.data.test.rating)
        return np.sqrt(((yt-yp)**2).mean())

    
class ContentBased(RecSys):
    def __init__(self,data):
        super().__init__(data)
        self.data=data
        self.Mm = self.calc_movie_feature_matrix()  
        
    def calc_movie_feature_matrix(self):
        """
        Create movie feature matrix in a numpy array of shape (#allmovies, #genres) 
        """
        movie_features = np.zeros((len(self.allmovies), len(self.genres)))
        for i, movie_id in enumerate(self.allmovies):
            movie_row = self.mid2idx[movie_id]
            movie_genres = self.data.movies.iloc[movie_row][self.genres]
            movie_features[i] = np.array(movie_genres)
        return movie_features
        
        
    
    def calc_item_item_similarity(self):
        """
        Create item-item similarity using Jaccard similarity
        """
        # Update the sim matrix by calculating item-item similarity using Jaccard similarity
        # Jaccard Similarity: J(A, B) = |A∩B| / |A∪B| 
        # your code here
        for i in range(len(self.allmovies)):
            for j in range(i, len(self.allmovies)):
                if i == j:
                    self.sim[i, j] = 1.0
                else:
                    genres_i = set(np.where(self.Mm[i] == 1)[0])
                    genres_j = set(np.where(self.Mm[j] == 1)[0])
                    intersection = genres_i.intersection(genres_j)
                    union = genres_i.union(genres_j)
                    jaccard_sim = len(intersection) / len(union)
                    self.sim[i, j] = jaccard_sim
                    self.sim[j, i] = jaccard_sim
                
class Collaborative(RecSys):    
    def __init__(self,data):
        super().__init__(data)
        
    def calc_item_item_similarity(self, simfunction, *X):  
        """
        Create item-item similarity using similarity function. 
        X is an optional transformed matrix of Mr
        """    
        # General function that calculates item-item similarity based on the sim function and data inputed
        if len(X)==0:
            self.sim = simfunction()            
        else:
            self.sim = simfunction(X[0]) # *X passes in a tuple format of (X,), to X[0] will be the actual transformed matrix
            
            
    def cossim(self):    
        """
        Calculates item-item similarity for all pairs of items using cosine similarity (values from 0 to 1) 
        on utility matrix. 
        
        Returns a cosine similarity matrix of size (#all movies, #all movies)
        
        Both Boundary checks (Step 5, 7) are important.
        """
        
        # MV_array_adujsted_zeros
        movie_ratings_allUsers = self.Mr.sum(axis=1)/(self.Mr > 0).sum(axis=1)
        
        # create a sparse matrix for operating cosine on its values
        movie_ratings_array = np.repeat(np.expand_dims(movie_ratings_allUsers, axis=1), self.Mr.shape[1], axis=1)
        
        # take care of all zero ratings (missing value/itentionally we don't know)
        MR_arr = self.Mr + (self.Mr==0) * movie_ratings_array - movie_ratings_array
        # print(MR_arr)
        
        # average all the ratings: divide by its magnitude!
        MR_arr_averaged = MR_arr/np.sqrt((MR_arr**2).sum(axis=0))
        
        # boundary check # 1: since dividing by magnitude may produce inf, zeros, etc.
        MR_arr_averaged[np.isnan(MR_arr_averaged)] = 0
        
        # item-item cosine sim
        # boundary check # 2: covariance/correlation values for A,A in diagonal is always 1.
        # - set diagonals explicility to 1
        cos = np.dot(MR_arr_averaged.T, MR_arr_averaged)
        for i in range(len(self.allmovies)):
            cos[i, i] = 1
        
        # Final cosine formula:
        # normalize - between 0-1s:
        self.sim =  0.5 + 0.5 * cos
    
    
    """def jacsim(self, Xr):
        
        #Calculates item-item similarity for all pairs of items using Jaccard similarity (values from 0 to 1)
        #Xr is the transformed rating matrix.
        
        jac_matrix = np.zeros((len(Xr.T), len(Xr.T)))
        #print(jac_matrix.shape)
        for i, ii in zip(Xr.T, enumerate(range(0,len(Xr.T)))):
            for j, jj in zip(Xr.T, enumerate(range(0,len(Xr.T)))):
                jac_matrix[ii,jj] = jaccard_score(i,j)
                
        return jac_matrix"""
        
    def jacsim(self, Xr):
        """
        Calculates item-item similarity for all pairs of items using Jaccard similarity (values from 0 to 1)
        Xr is the transformed rating matrix.
        """
        jaccard_similarity_matrix = np.zeros((Xr.shape[0], Xr.shape[0]))
        for i in range(0,Xr.shape[0]):
            for j in range(i+1,Xr.shape[1]):
                set_i = set(Xr[i])
                set_j = set(Xr[j])
                intersection = len(set_i.intersection(set_j))
                union = len(set_i.union(set_j))
                jaccard_similarity = intersection / union
                jaccard_similarity_matrix[i, j] = jaccard_similarity
                jaccard_similarity_matrix[j, i] = jaccard_similarity
                self.sim = jaccard_similarity_matrix

        # Convert the input matrix to binary matrix
        X_bin = Xr
        #print(X_bin[0])
        # Convert the binary matrix to sparse matrix format
        X_sparse = csr_matrix(X_bin)
        #X_sparse = np.nan_to_num(X_sparse)
        # Calculate the intersection matrix for items
        I = X_sparse.T.dot(X_sparse)
        I = np.nan_to_num(I)
        # Calculate the union matrix for items
        U = X_sparse.sum(axis=0) + X_sparse.sum(axis=0).T - I

        # Calculate the Jaccard similarity matrix for items
        J = I / U
        J = np.nan_to_num(J)
        J = np.asarray(J)
        return J

# Q1. Baseline models [15 pts]

### 1a. Complete the function `predict_everything_to_3` in the class `RecSys`  [5 pts]

In [27]:
# Creating Sample test data
np.random.seed(42)
sample_train = train[:30000]
sample_test = test[:30000]


sample_MV_users = MV_users[(MV_users.uID.isin(sample_train.uID)) | (MV_users.uID.isin(sample_test.uID))]
sample_MV_movies = MV_movies[(MV_movies.mID.isin(sample_train.mID)) | (MV_movies.mID.isin(sample_test.mID))]


sample_data = Data(sample_MV_users, sample_MV_movies, sample_train, sample_test)

In [28]:
# Sample tests predict_everything_to_3 in class RecSys

sample_rs = RecSys(sample_data)
sample_yp = sample_rs.predict_everything_to_3()
print(sample_rs.rmse(sample_yp))
assert sample_rs.rmse(sample_yp)==approx(1.2642784503423288, abs=1e-3), "Did you predict everything to 3 for the test data?"

1.2642784503423288


In [29]:
# Hidden tests predict_everything_to_3 in class RecSys
rs = RecSys(data)
yp = rs.predict_everything_to_3()
print(rs.rmse(yp))

1.2585510334053043


### 1b. Complete the function predict_to_user_average in the class RecSys [10 pts]
Hint: Include rated items only when averaging

In [30]:
# Sample tests predict_to_user_average in the class RecSys
sample_yp = sample_rs.predict_to_user_average()
print(sample_rs.rmse(sample_yp))
assert sample_rs.rmse(sample_yp)==approx(1.1429596846619763, abs=1e-3), "Check predict_to_user_average in the RecSys class. Did you predict to average rating for the user?" 

C:\Users\isaac\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\isaac\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


1.1429596846619763


In [31]:
# Hidden tests predict_to_user_average in the class RecSys
yp = rs.predict_to_user_average()
print(rs.rmse(yp))

1.0352910334228647


# Q2. Content-Based model [25 pts]

### 2a. Complete the function calc_movie_feature_matrix in the class ContentBased [5 pts]

In [32]:
cb = ContentBased(data)

In [33]:
# tests calc_movie_feature_matrix in the class ContentBased 
assert(cb.Mm.shape==(3883, 18))

### 2b. Complete the function calc_item_item_similarity in the class ContentBased [10 pts]
This function updates `self.sim` and does not return a value.    
Some factors to think about:     
1. The movie feature matrix has binary elements. Which similarity metric should be used?
2. What is the computation complexity (time complexity) on similarity calcuation?      
Hint: You may use functions in the `scipy.spatial.distance` module on the dense matrix, but it is quite slow (think about the time complexity). If you want to speed up, you may try using functions in the `scipy.sparse` module. 

In [34]:
cb.calc_item_item_similarity()

In [35]:
# Sample tests calc_item_item_similarity in ContentBased class 

sample_cb = ContentBased(sample_data)
sample_cb.calc_item_item_similarity() 

# print(np.trace(sample_cb.sim))
# print(sample_cb.sim[10:13,10:13])
assert(sample_cb.sim.sum() > 0), "Check calc_item_item_similarity."
assert(np.trace(sample_cb.sim) == 3152), "Check calc_item_item_similarity. What do you think np.trace(cb.sim) should be?"


ans = np.array([[1, 0.25, 0.],[0.25, 1, 0.],[0., 0., 1]])
for pred, true in zip(sample_cb.sim[10:13, 10:13], ans):
    assert approx(pred, 0.01) == true, "Check calc_item_item_similarity. Look at cb.sim"

In [36]:
# tests calc_item_item_similarity in ContentBased class 

In [37]:
# additional tests for calc_item_item_similarity in ContentBased class 

In [38]:
# additional tests for calc_item_item_similarity in ContentBased class

In [39]:
# additional tests for calc_item_item_similarity in ContentBased class

In [40]:
# additional tests for calc_item_item_similarity in ContentBased class

### 2c. Complete the function predict_from_sim in the class RecSys [5 pts]

In [41]:
# for a, b in zip(sample_MV_users.uID, sample_MV_movies.mID):
#     print(a, b, sample_cb.predict_from_sim(a,b))

# Sample tests for predict_from_sim in RecSys class 
assert(sample_cb.predict_from_sim(245,276)==approx(2.5128205128205128,abs=1e-2)), "Check predict_from_sim. Look at how you predicted a user rating on a movie given UserID and movieID."
assert(sample_cb.predict_from_sim(2026,2436)==approx(2.785714285714286,abs=1e-2)), "Check predict_from_sim. Look at how you predicted a user rating on a movie given UserID and movieID."

AssertionError: Check predict_from_sim. Look at how you predicted a user rating on a movie given UserID and movieID.

In [ ]:
# tests for predict_from_sim in RecSys class 

### 2d. Complete the function predict in the class RecSys [5 pts]
After completing the predict method in the RecSys class, run the cell below to calculate rating prediction and RMSE. How much does the performance increase compared to the baseline results from above? 

In [ ]:
# Sample tests method predict in the RecSys class 

sample_yp = sample_cb.predict()
sample_rmse = sample_cb.rmse(sample_yp)
print(sample_rmse)

assert(sample_rmse==approx(1.1962537249116723, abs=1e-2)), "Check method predict in the RecSys class."

In [ ]:
# Hidden tests method predict in the RecSys class 

yp = cb.predict()
rmse = cb.rmse(yp)
print(rmse)

In [ ]:
# tests method predict in the RecSys class 

# Q3. Collaborative Filtering

### 3a. Complete the function cossim in the class Collaborative [10 pts]
**To Do:**    
1.Impute the unrated entries in self.Mr to the user's average rating then subtract by the user mean, call this matrix X.   
2.Calculate cosine similarity for all item-item pairs. Don't forget to rescale the cosine similarity to be 0~1.    
You might encounter divide by zero warning (numpy will fill nan value for that entry). In that case, you can fill those with appropriate values.    

Hint: Let's say a movie item has not been rated by anyone. When you calculate similarity of this vector to anoter, you will get $\vec{0}$=[0,0,0,....,0]. When you normalize this vector, you'll get divide by zero warning and it will make nan value in self.sim matrix. Theoretically what should the similarity value for $\vec{x}_i \cdot \vec{x}_i$ when $\vec{x}_i = \vec{0}$? What about $\vec{x}_i \cdot \vec{x}_j$ when $\vec{x}_i = \vec{0}$ and $\vec{x}_j$ is an any vector?     

Hint: You may use `scipy.spatial.distance.cosine`, but it will be slow because its cosine function does vector-vector operation whereas you can implement matrix-matrix operation using numpy to calculate all cosines all at once (it can be 100 times faster than vector-vector operation in our data). Also pay attention to the definition. The scipy.spatial.distance provides distance, not similarity. 

3. Run the below cell that calculate yp and RMSE. 

In [ ]:
# Sample tests cossim method in the Collaborative class

sample_cf = Collaborative(sample_data)
sample_cf.calc_item_item_similarity(sample_cf.cossim)
sample_yp = sample_cf.predict()
sample_rmse = sample_cf.rmse(sample_yp)

assert(np.trace(sample_cf.sim)==3152), "Check cossim method in the Collaborative class. What should np.trace(cf.sim) equal?"
assert(sample_rmse==approx(1.1429596846619763, abs=5e-3)), "Check cossim method in the Collaborative class. rmse result is not as expected."
assert(sample_cf.sim[0,:3]==approx([1., 0.5, 0.5],abs=1e-2)), "Check cossim method in the Collaborative class. cf.sim isn't giving the expected results."

In [ ]:
#print(sample_cf.sim)
#print(sample_rmse)
#print(sample_cf.sim[0,:3])

In [43]:
# Hidden tests cossim method in the Collaborative class

cf = Collaborative(data)
cf.calc_item_item_similarity(cf.cossim)
yp = cf.predict()
rmse = cf.rmse(yp)
print(rmse)

C:\Users\isaac\AppData\Local\Temp\ipykernel_20800\3178567581.py:177: RuntimeWarning: invalid value encountered in divide
  MR_arr_averaged = MR_arr/np.sqrt((MR_arr**2).sum(axis=0))


1.1686969357092107


In [ ]:
# tests cossim method in the Collaborative class 

In [ ]:
# additional tests for cossim method in the Collaborative class

In [ ]:
# additional tests for cossim method in the Collaborative class

In [ ]:
# additional tests for cossim method in the Collaborative class

In [ ]:
# additional tests for cossim method in the Collaborative class

In [ ]:
# additional tests for cossim method in the Collaborative class

### 3b. Complete the function jacsim in the class Collaborative [15 pts]
**3b [15 pts] = 3b-i) [5 pts]+3b-ii) [5 pts]+ 3b-iii) [5 pts]**

Function `jacsim` calculates jaccard similarity between items using collaborative filtering method. When we have a rating matrix `self.Mr`, the entries of Mr matrix are 0 to 5 (0: unrated, 1-5: rating). We are interested to see which threshold method works better when we use jaccard dimilarity in the collaborative filtering.    
We may treat any rating 3 or above to be 1 and the negatively rated (below 3) and no-rating as 0. Or, we may treat movies with any ratings to be 1 and ones that has no rating as 0. In this question, we will complete a function jacsim that takes a transformed rating matrix X and calculate and returns a jaccard similarity matrix.     
Let's consider these input cases for the utility matrix $M_r$ with ratings 1-5 and 0s for no-rating.    
1. $M_r \geq 3$ 
2. $M_r \geq 0$ 
3. $M_r$, no transform.

Things to think about: 
- The cases 1 and 2 are straightforward to calculate Jaccard, but what does Jaccard mean for multicategory data?
- Time complexity: The matrix $M_r$ is much bigger than the item feature matrix $M_m$, therefore it will take very long time if we calculate on dense matrix.     
Hint: Use sparse matrix.
- Which method will give the best performance?

### 3b-i)  When $M_r\geq3$ [5 pts]
After you've implemented the jacsim function, run the code below. If implemented correctly, you'll have RMSE below 0.99. 

In [44]:
cf = Collaborative(data)
Xr = cf.Mr>=3
t0=time.perf_counter()
cf.calc_item_item_similarity(cf.jacsim,Xr)
t1=time.perf_counter()
time_sim = t1-t0
print('similarity calculation time',time_sim)
yp = cf.predict()
rmse = cf.rmse(yp)
print(rmse)
assert(rmse<0.99)

KeyboardInterrupt: 

In [ ]:
# tests RMSE for jacsim implementation

In [ ]:
# additional tests for RMSE for jacsim implementation

In [ ]:
# additional tests for jacsim implementation

In [ ]:
# additional tests for jacsim implementation

### 3b-ii)  When $M_r\geq1$ [5 pts]
After you've implemented the jacsim function, run the code below. If implemented correctly, you'll have RMSE below 1.0. 

In [ ]:
cf = Collaborative(data)
Xr = cf.Mr>=1
t0=time.perf_counter()
cf.calc_item_item_similarity(cf.jacsim,Xr)
t1=time.perf_counter()
time_sim = t1-t0
print('similarity calculation time',time_sim)
yp = cf.predict()
rmse = cf.rmse(yp)
print(rmse)
assert(rmse<1.0)

In [ ]:
# tests RMSE for jacsim implementation 

In [ ]:
# tests RMSE for jacsim implementation

In [ ]:
# tests jacsim implementation

In [ ]:
# tests performance of jacsim implementation

### 3b-iii)  When $M_r$; no transform [5 pts]
After you've implemented the jacsim function, run the code below. If implemented correctly, you'll have RMSE below 0.96

In [ ]:
cf = Collaborative(data)
Xr = cf.Mr.astype(int)
t0=time.perf_counter()
cf.calc_item_item_similarity(cf.jacsim,Xr)
t1=time.perf_counter()
time_sim = t1-t0
print('similarity calculation time',time_sim)
yp = cf.predict()
rmse = cf.rmse(yp)
print(rmse)
assert(rmse<0.96)

In [ ]:
# tests jacsim implementation RMSE

In [ ]:
# tests jacsim implementation RMSE

In [ ]:
# tests jacsim implementation

In [ ]:
# tests jacsim implementation performance

### 3.C Discussion [Peer Review]
Answer the questions below in this week's Peer Review assignment. <br>
1. Summarize the methods and performances: Below is a template/example.

|Method|RMSE|
|:----|:--------:|
|Baseline, $Y_p$=3| |
|Baseline, $Y_p=\mu_u$| |
|Content based, item-item| |
|Collaborative, cosine| |
|Collaborative, jaccard, $M_r\geq 3$|  |
|Collaborative, jaccard, $M_r\geq 1$|  |
|Collaborative, jaccard, $M_r$|  |

2. Discuss which method(s) work better than others and why.

|Method|RMSE|
|:----|:--------:|
|Baseline, $Y_p$=3| 1.2642784503423288 |
|Baseline, $Y_p=\mu_u$| 1.0352910334228647 |
|Content based, item-item| 1.1966092802947408|
|Collaborative, cosine| 1.0263081874204125 |
|Collaborative, jaccard, $M_r\geq 3$| 0.9819500134453443 |
|Collaborative, jaccard, $M_r\geq 1$| 1.0352910334228647 |
|Collaborative, jaccard, $M_r$| 0.9594067865 |

2. Discuss which method(s) work better than others and why.

I know my Jaccard Similarity matrix is incorrect.  I also think that the Jaccard Matrix using collaborative is the best of the bunch.  With an RMSE of less than .96 in some cases, the error is the least for Jaccard.  I think that collaberative based filtering works the best because it looks at actual user based data and not based on arbitrary facts about categories.  That allows for bad movies to be filtered down.  I also think the difference in Jaccard Similarity scores is caused by different cutoff values.

The cutoff of 1 just says weather a person watched the movie or not.  3 is a boolean of how much they liked it.  No cutoff gives data of scores which is the most relivant data.

In [ ]:
"""cf = Collaborative(data)
Xr = cf.Mr>=3
print(Xr.shape)

cf.calc_item_item_similarity(cf.jacsim,Xr)
#yp = cf.predict()
#rmse = cf.rmse(yp)
print(rmse)"""

In [ ]:
"""# Example sparse matrix with non-boolean factors
sparse_matrix = csr_matrix([[1, 0, 3, 0],
                            [0, 2, 0, 1],
                            [0, 1, 3, 0]])

# Convert the sparse matrix to sets
set_A = set(sparse_matrix[0].nonzero()[1])
set_B = set(sparse_matrix[1].nonzero()[1])

# Calculate the Jaccard similarity
intersection = set_A.intersection(set_B)
union = set_A.union(set_B)
jaccard_similarity = len(intersection) / len(union)

print("Jaccard similarity:", jaccard_similarity)"""